In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn import preprocessing
pd.options.display.max_colwidth = 100

In [2]:
TRAIN_PATH = 'train_dataset_RUTUBE/'
MODELS_PATH = 'doc2vec_models'

In [3]:
data =  pq.read_table(TRAIN_PATH+'videos.parquet').to_pandas()
emotions = pd.read_csv(TRAIN_PATH+'emotions.csv')

In [78]:
# schema = pa.schema([
#     ('item_id', pa.string()),
#     ('video_title', pa.string()),
#     ('author_title', pa.string()),
#     ('tv_title', pa.string()),
#     ('season', pa.int16()),
#     ('video_description', pa.string()),
#     ('category_title', pa.string()),
#     ('publicated', pa.string()),
#     ('duration', pa.int32()),
#     ('channel_sub', pa.int32()),
#     ('tv_sub', pa.int32()),
#     ('CTR_10days_21_07', pa.float32()),
#     ('CTR_10days_01_08', pa.float32()),
#     ('CTR_10days_10_08', pa.float32()),
#     ('CTR_10days_21_08', pa.float32()),
#     ('__index_level_0__', pa.int64())
# ])
#
# source_table = pa.Table.from_pandas(data)
# source_table = source_table.cast(schema)
# pq.write_table(source_table, 'my_parquet_table.parquet')

In [8]:
data.rename(columns={'ctr.CTR_10days_21_07': 'CTR_10days_21_07', 'ctr.CTR_10days_01_08': 'CTR_10days_01_08',
                     'ctr.CTR10days_10_08': 'CTR_10days_10_08', 'ctr.CTR_10days_21_08': 'CTR_10days_21_08'}, inplace=True)

In [2]:
users = pq.read_table('train_dataset_RUTUBE/player_starts_train.parquet').to_pandas()

In [19]:
small_users = pd.read_csv('train_dataset_RUTUBE/small_player_starts_train.csv')

In [191]:
import string
translator = str.maketrans('', '', string.punctuation)

data['video_title'] = data['video_title'].apply(lambda x: x.translate(translator).lower().strip() if isinstance(x, str) else x)
data['video_description'].fillna("", inplace=True)
data['video_description'] = data['video_description'].apply(lambda x: x.translate(translator).lower().strip() if isinstance(x, str) else x)


In [193]:
video_descriptions = [TaggedDocument(doc, [i]) for i, doc in enumerate(data['video_description'].tolist())]
author_titles = [TaggedDocument(doc, [i]) for i, doc in enumerate(data['author_title'].tolist())]
video_titles = [TaggedDocument(doc, [i]) for i, doc in enumerate(data['video_title'].tolist())]

model_video_descr = Doc2Vec(video_descriptions,
                vector_size=100,
                window=len(video_descriptions),
                workers=16,
                epochs=10,
                seed=42)

model_author_titles = Doc2Vec(author_titles,
                vector_size=100,
                window=len(author_titles),
                workers=8,
                epochs=10,
                seed=42)

model_video_titles = Doc2Vec(video_titles,
                vector_size=100,
                window=len(video_titles),
                workers=8,
                epochs=10,
                seed=42)

KeyboardInterrupt: 

In [ ]:
# model_video_titles.save(MODELS_PATH+'model_video_titles')
# model_author_titles.save(MODELS_PATH+'model_author_titles')
model_video_descr.save(MODELS_PATH+'model_video_descr')


In [4]:
# author_titles = np.load('doc2vec_models/model_author_titles.dv.vectors.npy', allow_pickle=True)
video_titles = np.load('doc2vec_models/model_video_titles.dv.vectors.npy', allow_pickle=True)
video_descrs = np.load('doc2vec_models/model_video_descr_10epochs.dv.vectors.npy', allow_pickle=True)

In [5]:
video_titles_normalized = preprocessing.minmax_scale(video_titles.T).T
video_descrs_normalized = preprocessing.minmax_scale(video_descrs.T).T

In [6]:
text_embs = np.concatenate([video_titles_normalized,video_descrs_normalized], axis=-1)

In [7]:
cols_titles = [f'vid_title_{i}' for i in list(map(str, list(range(100))))]
cols_descrs_ = [f'vid_descr_{i}' for i in list(map(str, list(range(100))))]
cols_titles.extend(cols_descrs_)
df = pd.DataFrame(text_embs, columns = cols_titles)

In [8]:
df['item_id'] = data.item_id
cols = ['item_id']
cols.extend(cols_titles)
df = df[cols]

In [9]:
# categories_dict = {cat: idx for idx, cat in enumerate(data['category_title'].unique())}
# data['category_title'] = data['category_title'].map(categories_dict)
author_titles = {title: idx for idx, title in enumerate(data['author_title'].unique())}
data['author_title'] = data['author_title'].map(author_titles)

In [10]:
df = pd.concat([df, pd.get_dummies(data['category_title'], prefix='cat')], axis=1)

In [233]:
# videos_emotions_types = emotions.groupby(['item_id', 'type']).size().unstack(fill_value=0).reset_index()

In [11]:
final = df.merge(data,how='inner', on='item_id')

In [97]:
# final.drop(['video_title', 'tv_title', 'season', 'video_description',
#        'channel_sub', 'tv_sub',
#        ], inplace=True, axis=1)

In [15]:
final['ctr.CTR_10days_21_07'].fillna(data['ctr.CTR_10days_21_07'].mode()[0], inplace=True)
final['ctr.CTR_10days_01_08'].fillna(data['ctr.CTR_10days_01_08'].mode()[0], inplace=True)
final['ctr.CTR_10days_10_08'].fillna(data['ctr.CTR_10days_10_08'].mode()[0], inplace=True)
final['ctr.CTR_10days_21_08'].fillna(data['ctr.CTR_10days_21_08'].mode()[0], inplace=True)

In [2]:
final.head()


NameError: name 'final' is not defined

In [1]:
final['tv_title'].dropna(inplace=True)

NameError: name 'final' is not defined

In [16]:
final['publicated'] = pd.to_datetime(final['publicated'])

In [17]:
final['upld_year'] = final['publicated'].apply(lambda x: x.year)
final['upld_month'] = final['publicated'].apply(lambda x: x.month)
final['upld_day'] = final['publicated'].apply(lambda x: x.day)
final['upld_hour'] = final['publicated'].apply(lambda x: x.hour)
final['upld_minute'] = final['publicated'].apply(lambda x: x.minute)
final['upld_second'] = final['publicated'].apply(lambda x: x.second)
final['upld_dayofweek'] = final['publicated'].apply(lambda x: x.weekday())

In [1]:
# final = final[final.columns.drop(list(df.filter(regex='vid_title')))]

NameError: name 'final' is not defined

In [19]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(final, test_size=0.2, random_state=42)

In [21]:
import pynndescent
index = pynndescent.NNDescent(train.drop(['item_id', 'video_description', 'tv_title', 'video_title', 'ctr.CTR_10days_01_08', 'ctr.CTR_10days_21_07', 'ctr.CTR_10days_10_08', 'ctr.CTR_10days_21_08', 'tv_sub', 'season', 'publicated', 'category_title'], axis=1), n_jobs=8, n_neighbors=5, metric='jaccard')

In [22]:
index.prepare()

/Users/gd/lib/python3.10/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [23]:
import pickle
with open('final_model_jaccard_2.pkl', 'wb') as f:
    pickle.dump(index, f)

In [24]:
final.to_csv('final_df_2.csv')

In [26]:
neighbors = index.query(test.drop(['item_id', 'video_description', 'tv_title', 'video_title', 'ctr.CTR_10days_01_08', 'ctr.CTR_10days_21_07', 'ctr.CTR_10days_10_08', 'ctr.CTR_10days_21_08', 'tv_sub', 'season', 'publicated', 'category_title'], axis=1).head(10), k=100)

In [27]:
pickled_model = pickle.load(open('final_model_jaccard_2.pkl', 'rb'))

In [32]:
neighbors =  pickled_model.query(test.drop(['item_id', 'video_description', 'tv_title', 'video_title', 'ctr.CTR_10days_01_08', 'ctr.CTR_10days_21_07', 'ctr.CTR_10days_10_08', 'ctr.CTR_10days_21_08', 'tv_sub', 'season', 'publicated', 'category_title'], axis=1).head(10), k=100)

In [33]:
for video_id in test.head(10)['item_id']:
    print(data[data['item_id'] == video_id].video_title)

1025985    Арай Чобанян показал палату в которой рожает Ира Пинчук
Name: video_title, dtype: object
468857    LAUNCH ● X-DIAG PRO3 и DIAGZONE PRO ● АКТИВАЦИЯ ВСЕХ МАРОК АВТО
Name: video_title, dtype: object
1638168    1992  Реклама Барби Маттел Голливудские Волосы  Barbie Hollywood Hair
Name: video_title, dtype: object
423764    АСМР Карта Нурсултана в Казахстане ??
Name: video_title, dtype: object
1597718    Micro Machines (NES Dendy 8bit) - Full Walkthrough Longplay no commentary - Микро машинки на денди
Name: video_title, dtype: object
1698655    Кинетический песок АСМР • Видео для расслабления • ASMR выпуск 4
Name: video_title, dtype: object
67347    Зефирантес Выскочка_Расцвел Зефирантес_Красота на моем окне.Выскочка цветок уход.
Name: video_title, dtype: object
872391    «Повышенная государственная академическая стипендия за достижения»
Name: video_title, dtype: object
1564647    Вестница весны - под музыку, для любителей просто наблюдать за процессом
Name: video_title, dtype: ob

In [34]:
for i in neighbors[0][1]:
    print(data[data['item_id'] == train.iloc[i].item_id].video_title)

1823170    ? Девушки на Мотоциклах ! Красивейший Бернаут ? !
Name: video_title, dtype: object
1409011    Разварки на ниву
Name: video_title, dtype: object
881064    КАК ПРАВИЛЬНО настроить тормоза на электросамокате?
Name: video_title, dtype: object
1945673    Как отдыхают водители, бурятский бухлëр, в профессию без гражданства РФ, получаю трос
Name: video_title, dtype: object
2198594    Toyota Camry 2AZ-FE раскоксовка димексидом (Омск).mp4
Name: video_title, dtype: object
552280    форд фьюжн замена втулок стабилизатора, снятие подрамника!
Name: video_title, dtype: object
677480    Сервис чуть не угробил двигатель
Name: video_title, dtype: object
1345465    Renault Premium DXI 11 шум ДВС.mp4
Name: video_title, dtype: object
121841    ТОРМОЗА НА ВОДОПРОВОДНЫХ ШЛАНГАХ - ЧТО БУДЕТ ?
Name: video_title, dtype: object
198988    Замена сальника задней ступицы на ГАЗели. Сальник полуоси.
Name: video_title, dtype: object
1168901    Подборка ЖЕСТЬ на СТО
Name: video_title, dtype: object
574511 

In [297]:
import pickle

pickle.dump(index, open('index_vid_titles_categs.pkl', 'wb'))
# m = pickle.load(open('index_vid_titles.pkl', 'rb'))

In [ ]:

def wid_long(videos: pd.DataFrame, user_id: int, users: pd.DataFrame) -> pd.DataFrame:
    tqdm.pandas()
    hist = users[users["user_id"] == user_id]
    hist = pd.merge(hist, videos, on='item_id').drop(columns=["user_id"], axis=1)
    for i in tqdm(range(100)):
        hist[f"v_title_{i}"] = hist.progress_apply(lambda row: row[f"v_title_{i}"] * 2
        if ((row["watch_time"] / (row["duration"] / 1000)) > 0.25
            if (row["duration"] / 1000) > 300
            else row["watch_time"] > 30)
        else 1, axis=1)
    return hist.drop(columns=["watch_time", "duration"], axis=1)


def like(videos: pd.DataFrame, user_id: int, users: pd.DataFrame, emotions: pd.DataFrame) -> pd.DataFrame:
    emotions = emotions[["C2", "C3", "C4"]]
    emotions = emotions[emotions["C2"] == user_id]
    emotions = emotions[["C3", "C4"]]
    hist = users[users["user_id"] == user_id]
    hist = pd.merge(hist, videos, on='item_id').drop(columns=["user_id"], axis=1)
    hist = pd.merge(hist, emotions, left_on='item_id', right_on="C3").drop(columns=["user_id"], axis=1)
    for i in tqdm(range(100)):
        hist[f"v_title_{i}"] = hist.progress_apply(lambda row: row[f"v_title_{i}"] * 2 if row["C4"] == "pos_emotions"
        else (0.5 if row["C4"] == "neg_emotions" else 1), axis=1)
    return hist.drop(columns=["C4", "C3"], axis=1)


def encode_categories(videos: pd.DataFrame) -> pd.DataFrame:
    # получаем dummy-переменные
    dummies = pd.get_dummies(videos['category_title'], prefix='cat')

    # объединяем с исходным датафреймом
    videos = pd.concat([videos, dummies], axis=1)
    return videos.drop(columns=["category_title"], axis=1)

In [170]:
data

,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,CTR_10days_21_07,CTR_10days_01_08,CTR_10days_10_08,CTR_10days_21_08
0,video_165654,MSI Pro MP241X недоОБЗОР (РЕШЕНИЕ ПРОБЛЕМЫ С МЕРЦАНИЕМ ПРИ ВЫКЛЮЧЕННОМ ПК),0,None,0,В видео я обывательским взглядом расскажу про монитор MSI Pro MP241X. Так же покажу решение проб...,Технологии и интернет,2022-12-08 13:53:05+03:00,391382,0,0,NaN,0.0,0.000000,NaN
1,video_1173704,Наложение пястно фаланговой повязки на кисть,1,None,0,Видео с канала УЦ Академия безопасности (ab-dpo.ru): Наложение пястно фаланговой повязки на кисть,Образование,2022-03-24 09:19:15+03:00,125922,26,0,NaN,0.0,0.000000,0.00
2,video_23927,SilverstoneF1 Sochi Pro и Neoline x cop 6000s 5900s тест на радар СКАТ,2,None,0,SilverstoneF1 Sochi Pro и Neoline x cop 6000s 5900s тест на радар СКАТ\n#антирадар #радардетекто...,Авто-мото,2022-03-19 17:41:49+03:00,436570,2,0,NaN,NaN,0.000000,0.00
3,video_1003780,БОЛЬНИЦА в Brookhaven! ДОКТОР ПУПКИН спас ЖЕНИХА! ТИК ТОКЕР НА СВАДЬБЕ! Сериал БРУКХЭВЭН 1 серия!,3,None,0,Играем в Роблокс (Roblox) - БОЛЬНИЦА в Brookhaven! ДОКТОР ПУПКИН спас ЖЕНИХА! ТИК ТОКЕР НА СВАДЬ...,Детям,2021-02-20 11:50:53+03:00,719377,673,0,0.0,1.0,0.000000,0.00
4,video_105383,"Вебинар ""Особенности трудоустройства граждан Белоруссии, Казахстана, Киргизии и Армении""",4,None,0,"10.08.2023 Вебинар ""Особенности трудоустройства граждан Белоруссии, Казахстана, Киргизии и Армен...",Бизнес и предпринимательство,2023-08-11 09:02:07+03:00,3834404,19,0,0.0,0.0,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320655,video_839160,Любимцы: Гена в памперсе бегает по клинике,147463,Любимцы,1,None,Сериалы,2017-09-11 22:00:13+03:00,23290,0,0,NaN,0.0,0.333333,0.25
2320656,video_2134427,"Телеканал «СПАС», «Главное», интервью С. В. Степашина",34203,None,0,None,Интервью,2022-03-28 11:19:33+03:00,1548792,57,0,0.0,0.0,0.000000,0.00
2320657,video_1453482,Взятие 3 отметок на ИС 7,155222,None,0,None,Видеоигры,2022-09-10 20:05:17+03:00,520001,12,0,NaN,NaN,NaN,NaN
2320658,video_2225207,Простой и вкусный рецепт пирога с грушами и сыром камамбер (рецепт в комментариях),1646,None,0,None,Разное,2023-07-06 12:16:29+03:00,8267,9,0,0.0,0.0,0.000000,0.00


In [172]:
final.to_csv('final_df.csv', index=False)

In [173]:
final

,item_id,vid_title_0,vid_title_1,vid_title_2,vid_title_3,vid_title_4,vid_title_5,vid_title_6,vid_title_7,vid_title_8,...,CTR_10days_01_08,CTR_10days_10_08,CTR_10days_21_08,upld_year,upld_month,upld_day,upld_hour,upld_minute,upld_second,upld_dayofweek
0,video_165654,0.317030,0.583350,0.319950,0.233488,0.397311,0.335484,0.683385,0.544738,0.459705,...,0.0,0.000000,0.00,2022,12,8,13,53,5,3
1,video_1173704,0.481264,0.387798,0.602036,0.802317,0.655285,0.626316,0.562532,0.271270,0.515960,...,0.0,0.000000,0.00,2022,3,24,9,19,15,3
2,video_23927,0.648496,0.440843,0.160715,0.613938,0.367600,0.542256,0.304734,0.719216,0.541566,...,0.0,0.000000,0.00,2022,3,19,17,41,49,5
3,video_1003780,0.203357,0.307920,0.590847,0.516022,0.755484,0.283917,0.717133,0.625334,0.525521,...,1.0,0.000000,0.00,2021,2,20,11,50,53,5
4,video_105383,0.744297,0.835887,0.621679,0.773973,0.504829,0.539626,0.493591,0.560223,0.590560,...,0.0,0.000000,0.00,2023,8,11,9,2,7,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2320657,video_839160,0.292854,0.666169,0.771124,0.671644,0.643341,0.410232,0.162509,0.522382,0.561864,...,0.0,0.333333,0.25,2017,9,11,22,0,13,0
2320658,video_2134427,0.557534,0.428624,0.511768,0.810072,0.586449,0.477702,0.389940,0.329786,0.467130,...,0.0,0.000000,0.00,2022,3,28,11,19,33,0
2320659,video_1453482,0.434726,0.334742,0.281761,0.154127,0.254287,0.386525,0.505897,0.527292,0.430226,...,0.0,0.000000,0.00,2022,9,10,20,5,17,5
2320660,video_2225207,0.805957,0.767042,0.671743,0.632880,0.815049,0.631796,0.608704,0.433524,0.722830,...,0.0,0.000000,0.00,2023,7,6,12,16,29,3
